# Smolagents (Hugging Face)

**Objectifs**  
- Découvrir les **CodeAgent** et **ToolCallingAgent** avec **smolagents**.  
- Utiliser un modèle **OpenAI** via `OpenAIServerModel` (vous avez `OPENAI_API_KEY`).  
- Construire 2 outils : `calculator` (arithmétique) et `kb_lookup` (mini base locale).  
- Comparer **CodeAgent** vs **ToolCallingAgent** sur 2–3 tâches.

> Docs utiles : guided tour, outils, modèles (OpenAI).

## 0) Installation & Setup

In [ ]:
import os

# Try to load from .env file if available (optional)
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  # dotenv not installed, that's okay

assert os.getenv("OPENAI_API_KEY"), "⚠️ Please set OPENAI_API_KEY in your environment!"
print("✅ OK: OPENAI_API_KEY detected")

## 1) Choisir le modèle (OpenAI)

In [ ]:
from smolagents import OpenAIServerModel

model = OpenAIServerModel(
    model_id="gpt-4o-mini",
    api_base="https://api.openai.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
)
print(model, dir(model))

## 2) Créer des outils (@tool)

In [ ]:
from smolagents import tool
import re

@tool
def calculator(expr: str) -> str:
    """
    Évalue une expression arithmétique simple.

    Args:
        expr (str): Expression arithmétique à évaluer. Autorisés: chiffres, espaces, point décimal,
                    et opérateurs +, -, *, / ainsi que les parenthèses ( et ).
    """
    if not re.fullmatch(r"[0-9+\-*/()\s\.]+", expr):
        return "Expression non autorisée."
    try:
        return str(eval(expr, {"__builtins__": {}}, {}))
    except Exception as e:
        return f"Erreur: {e}"


In [ ]:
KB = {
  "jbv": "Jambon-beurre Vegan",
  "mtmgapd": "Mange ta main et garde l'autre pour demain",
}

@tool
def kb_lookup(term: str) -> str:
    """
    Retourne une petite définition depuis la KB locale (clé exacte).

    Args:
        term (str): clé exacte à récupérer dans la KB locale

    Returns:
        str: définition associé au term
    """
    return KB.get(term.lower(), f"(inconnue) clés={list(KB)})")

## 3) CodeAgent — planifier et agir en écrivant du *code*

In [ ]:
from smolagents import CodeAgent
agent_code = CodeAgent(tools=[calculator, kb_lookup], model=model, add_base_tools=False)
agent_code.run("Calcule 37*19, puis définis 'mtmgapd' en une phrase")

## 4) ToolCallingAgent — *tool calls* JSON natifs du modèle

In [ ]:
from smolagents import ToolCallingAgent
agent_tool = ToolCallingAgent(tools=[calculator, kb_lookup], model=model, add_base_tools=False)
agent_tool.run("Calcule (12+5)*9, puis définis 'mtmgapd' en une phrase")

## 5) Mini comparaison qualitative

In [ ]:
prompts = [
  "Calcule 23*51 et donne une phrase d'explication.",
  "Donne la définition de 'mtmgapd' et un exemple très court.",
]
for p in prompts:
    print("\n=== Prompt ===", p)
    print("\n--- CodeAgent ---")
    print(agent_code.run(p))
    print("\n--- ToolCallingAgent ---")
    print(agent_tool.run(p))

## 6) Questions
1. Différence conceptuelle entre **CodeAgent** et **ToolCallingAgent** ?  
2. Quand préféreriez-vous l’un ou l’autre ? (indices : sandbox, latence, traçabilité)  
3. Où placeriez-vous un **WebSearchTool** ou un **RAG** dans ce TP ?

## 🔎 Bonus — Sandbox & WebSearchTool (optionnel)

In [ ]:
# Indices:
# - add_base_tools=True ajoute un interpréteur Python sandboxé & web search (DuckDuckGo) selon l'env.
# - Pour un vrai sandbox, voir la doc smolagents (E2B/Modal/Docker).
from smolagents import CodeAgent
agent_code2 = CodeAgent(tools=[kb_lookup], model=model, add_base_tools=True)
agent_code2.run("Quel est le produit de 91*97 puis fais une recherche web sommaire sur 'RAG' ?")